<a href="https://colab.research.google.com/github/ehabsameh/Final_Project/blob/main/SnapChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#import needed packages
!pip install pyjanitor==0.23.1
import pandas as pd
import json
from urllib.request import urlopen
import matplotlib.pyplot as plt
import plotly.express as px
from urllib import request
import json
import numpy as np
import seaborn as sns
import janitor
import datetime
import statsmodels.api as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
import plotly.graph_objects as go
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objs as go
from fbprophet import Prophet

pd.options.display.max_columns = None

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pystan~=2.14
!pip install fbprophet

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path='/content/drive/MyDrive/AWS_Final_Project/usage_pre_covid_nordic.csv'
snap = pd.read_csv(path)

In [15]:
snapA = snap.groupby('event_date')['APP_APPLICATION_OPEN_UU'].sum().reset_index()

In [100]:

############ Question 1

train_data = snap.groupby('event_date')['APP_APPLICATION_OPEN_UU'].sum().reset_index().rename(columns={'event_date': 'ds', 'APP_APPLICATION_OPEN_UU': 'y'})
train_data['ds'] = pd.to_datetime(train_data['ds'])

# Select certain number of historical days
#train_data = train_data[train_data['ds'] > train_data['ds'].max() - pd.Timedelta(days=40)]
# lockdown period
lockdowns = pd.DataFrame([
    {'holiday': 'lockdown_1', 'ds': '2020-03-13', 'lower_window': 0, 'ds_upper': '2020-05-29'}
])
for t_col in ['ds', 'ds_upper']:
    lockdowns[t_col] = pd.to_datetime(lockdowns[t_col])
lockdowns['upper_window'] = (lockdowns['ds_upper'] - lockdowns['ds']).dt.days
lockdowns



# forecast model
m1 = Prophet(
   changepoint_prior_scale=0.01,
     #            changepoints=['2020-03-13'],
     n_changepoints=10,
      #     seasonality_prior_scale = 0.8,
            holidays = lockdowns,
       #      , weekly_seasonality=False
       yearly_seasonality= True
             )
# pre and post covid seasonality
train_data = train_data.copy()
train_data['pre_covid'] = pd.to_datetime(train_data['ds']) < pd.to_datetime('2020-03-13')
train_data['post_covid'] = ~train_data['pre_covid']

m1.add_seasonality(
    name='weekly_pre_covid',
    period=7,
    fourier_order=3,
    condition_name='pre_covid',
)
m1.add_seasonality(
    name='weekly_post_covid',
    period=7,
    fourier_order=3,
    condition_name='post_covid',
);

# fit model
m1 = m1.fit(train_data)

future1 = m1.make_future_dataframe(periods=101)
future1['pre_covid'] = pd.to_datetime(future1['ds']) < pd.to_datetime('2020-03-13')
future1['post_covid'] = ~future1['pre_covid']

# predict
forecast1 = m1.predict(future1)

# forecast plot
plot_plotly(m1, forecast1)
#plot_components_plotly(m1, forecast1)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [96]:
future1

,ds,pre_covid,post_covid
0,2019-01-01,True,False
1,2019-01-02,True,False
2,2019-01-03,True,False
3,2019-01-04,True,False
4,2019-01-05,True,False
...,...,...,...
542,2020-06-26,False,True
543,2020-06-27,False,True
544,2020-06-28,False,True
545,2020-06-29,False,True


In [94]:
train_data = snap.groupby('event_date')['APP_APPLICATION_OPEN_UU'].sum().reset_index().rename(columns={'event_date': 'ds', 'APP_APPLICATION_OPEN_UU': 'y'})
train_data['ds'] = pd.to_datetime(train_data['ds'])

# lockdown period
lockdowns = pd.DataFrame([
    {'holiday': 'lockdown_1', 'ds': '2020-03-13', 'lower_window': -15, 'upper_window': 0}
])
for t_col in ['ds']:
    lockdowns[t_col] = pd.to_datetime(lockdowns[t_col])

# forecast model
m1 = Prophet(changepoint_prior_scale=0.01,
             seasonality_prior_scale=0.8,
             holidays=lockdowns,
             weekly_seasonality=False)

# Add a weekly seasonality component for the last 9 days of data
train_data['last_9_days'] = (train_data['ds'] >= train_data['ds'].max() - pd.Timedelta(days=9)).astype(int)
m1.add_seasonality(
    name='weekly_last_9_days',
    period=7,
    fourier_order=3,
    condition_name='last_9_days',
)

# fit model
m1 = m1.fit(train_data)

# make future dataframe
future1 = m1.make_future_dataframe(periods=101)

# Add a column for the last 9 days of data to the future dataframe
future1['last_9_days'] = (future1['ds'] >= train_data['ds'].max() - pd.Timedelta(days=9)).astype(int)

# predict
forecast1 = m1.predict(future1)

# forecast plot
plot_plotly(m1, forecast1)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.9/dist-packages/fbprophet/forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [86]:
train_data

,ds,y,pre_covid,post_covid
0,2019-01-01,7012787,True,False
1,2019-01-02,6522305,True,False
2,2019-01-03,6457837,True,False
3,2019-01-04,6529505,True,False
4,2019-01-05,6650731,True,False
...,...,...,...,...
441,2020-03-17,7110274,False,True
442,2020-03-18,7128569,False,True
443,2020-03-19,7167179,False,True
444,2020-03-20,7237818,False,True


In [80]:
lockdowns

,holiday,ds,lower_window,ds_upper,upper_window
0,lockdown_1,2020-03-13,0,2020-05-29,77


In [ ]:

m = Prophet(holidays=holidays)
m.add_country_holidays(country_name='US')
m.fit(df)
m.train_holiday_names


###

# generate the forecast for the next 9 days
future_dates = pd.date_range(start=train_data['ds'].max()+pd.Timedelta(days=1), periods=90, freq='D')
future_data = pd.DataFrame({'ds': future_dates})

forecast = model.predict(future_data)['yhat'].values

# create a new DataFrame with the forecasted values
forecast_df1 = pd.DataFrame({'event_date': future_dates, 'APP_APPLICATION_OPEN_UU': forecast})
forecast_df1['event_date'] = forecast_df1['event_date'].apply(lambda x: x.date().strftime('%Y-%m-%d'))


In [30]:

snapF = pd.concat([snapA, forecast_df1], ignore_index=True)


In [31]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=snapF['event_date'], y=snapF['APP_APPLICATION_OPEN_UU'], mode='lines', name='APP_APPLICATION_OPEN_UU'))
fig.update_layout(title='Forecasted Usage', xaxis_title='Date', yaxis_title='DAY')
fig.update_traces(hoverinfo='text', hovertext=[f"event_date: {date}<br>APP_APPLICATION_OPEN_UU: {APP_APPLICATION_OPEN_UU:.2f}" for date, APP_APPLICATION_OPEN_UU in zip(snapAF['event_date'], snapAF['APP_APPLICATION_OPEN_UU'])])
fig.show()

In [ ]:
!pip install rpy2
%load_ext rpy2.ipython


In [14]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA


train_data = snap.groupby('event_date')['APP_APPLICATION_OPEN_UU'].sum().reset_index().rename(columns={'event_date': 'date', 'APP_APPLICATION_OPEN_UU': 'usage'})
train_data['date'] = pd.to_datetime(train_data['date'])

# Select the last 14 days of data
train_data = train_data[train_data['date'] > train_data['date'].max() - pd.Timedelta(days=120)]

# fit an ARIMA model to the training data
model = ARIMA(train_data.set_index('date'), order=(1, 1, 0))
model_fit = model.fit()

# generate the forecast for the next 9 days
future_dates = pd.date_range(start=train_data['date'].max()+pd.Timedelta(days=1), periods=9, freq='D')
forecast = model_fit.forecast(steps=9)[0]

# create a new DataFrame with the forecasted values
forecast_df1 = pd.DataFrame({'event_date': future_dates, 'APP_APPLICATION_OPEN_UU': forecast})
forecast_df1['event_date'] = forecast_df1['event_date'].apply(lambda x: x.date().strftime('%Y-%m-%d'))





/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

